# Intuition for Text Generation

This notebook shows an end to end process for the objective of generating text on a character level. It first reads a text file as defined in `filename` and performs the following:

* Text Cleanup by lowercasing everything
* Text Cleanup by removing unwanted symbols
* Creating a mapping for character indices and indices to characters
* Defining x and y vectors for classification
* Pass it to a multi-layer perceptron
* Generate some text

## Required Libraries

* `nltk`
* `scikit-learn`
* `torch`

In [1]:
# import sys
# !{sys.executable} -m pip install torch

### Import the Necessary Libraries

In [2]:
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/cs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Read a Text File and Get Raw Text

You may change `filename` to the location of your own file.

In [3]:
filename = "names.txt"
f = open(filename, 'r')

raw_text = f.read()

### Examine First 1000 Characters

In [4]:
raw_text[0:1000]

'Michael\nChristopher\nJessica\nMatthew\nAshley\nJennifer\nJoshua\nAmanda\nDaniel\nDavid\nJames\nRobert\nJohn\nJoseph\nAndrew\nRyan\nBrandon\nJason\nJustin\nSarah\nWilliam\nJonathan\nStephanie\nBrian\nNicole\nNicholas\nAnthony\nHeather\nEric\nElizabeth\nAdam\nMegan\nMelissa\nKevin\nSteven\nThomas\nTimothy\nChristina\nKyle\nRachel\nLaura\nLauren\nAmber\nBrittany\nDanielle\nRichard\nKimberly\nJeffrey\nAmy\nCrystal\nMichelle\nTiffany\nJeremy\nBenjamin\nMark\nEmily\nAaron\nCharles\nRebecca\nJacob\nStephen\nPatrick\nSean\nErin\nZachary\nJamie\nKelly\nSamantha\nNathan\nSara\nDustin\nPaul\nAngela\nTyler\nScott\nKatherine\nAndrea\nGregory\nErica\nMary\nTravis\nLisa\nKenneth\nBryan\nLindsey\nKristen\nJose\nAlexander\nJesse\nKatie\nLindsay\nShannon\nVanessa\nCourtney\nChristine\nAlicia\nCody\nAllison\nBradley\nSamuel\nShawn\nApril\nDerek\nKathryn\nKristin\nChad\nJenna\nTara\nMaria\nKrystal\nJared\nAnna\nEdward\nJulie\nPeter\nHolly\nMarcus\nKristina\nNatalie\nJordan\nVictoria\nJacqueline\nCorey\n

In [5]:
names = raw_text.split("\n")
lengths = [len(name) for name in names]
max(lengths)

15

### Cleanup Text

* Lowercase all characters
* Remove special symbols

Note that this retains things like spaces, commas, periods and other punctuations.

In [6]:
processed_text = raw_text.lower()
processed_text = re.sub(r'[^\x00-\x7f]', r'', processed_text)
# processed_text = re.sub(r'\n', r'', processed_text)

processed_text

'michael\nchristopher\njessica\nmatthew\nashley\njennifer\njoshua\namanda\ndaniel\ndavid\njames\nrobert\njohn\njoseph\nandrew\nryan\nbrandon\njason\njustin\nsarah\nwilliam\njonathan\nstephanie\nbrian\nnicole\nnicholas\nanthony\nheather\neric\nelizabeth\nadam\nmegan\nmelissa\nkevin\nsteven\nthomas\ntimothy\nchristina\nkyle\nrachel\nlaura\nlauren\namber\nbrittany\ndanielle\nrichard\nkimberly\njeffrey\namy\ncrystal\nmichelle\ntiffany\njeremy\nbenjamin\nmark\nemily\naaron\ncharles\nrebecca\njacob\nstephen\npatrick\nsean\nerin\nzachary\njamie\nkelly\nsamantha\nnathan\nsara\ndustin\npaul\nangela\ntyler\nscott\nkatherine\nandrea\ngregory\nerica\nmary\ntravis\nlisa\nkenneth\nbryan\nlindsey\nkristen\njose\nalexander\njesse\nkatie\nlindsay\nshannon\nvanessa\ncourtney\nchristine\nalicia\ncody\nallison\nbradley\nsamuel\nshawn\napril\nderek\nkathryn\nkristin\nchad\njenna\ntara\nmaria\nkrystal\njared\nanna\nedward\njulie\npeter\nholly\nmarcus\nkristina\nnatalie\njordan\nvictoria\njacqueline\ncorey\n

### Create Index Mappings

* `character_indices`: Return the index of a given character. Example: `character_indices['a']`
* `indices_characters`: Returns the character given an index. Example: `indices_characters[0]`

In this case, `characters` serves as your **vocabulary**.

In [7]:
print("Corpus Length: {}".format(len(processed_text)))

characters = sorted(list(set(processed_text)))
print("Total Characters: {}".format(len(characters)))

character_indices = dict((c, i) for i,c in enumerate(characters))
indices_characters = dict((i, c) for i, c in enumerate(characters))

Corpus Length: 133558
Total Characters: 27


In [8]:
character_indices

{'\n': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [9]:
indices_characters

{0: '\n',
 1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z'}

### Convert to a Set of Symbols of Fixed Length

* `maxlen`: Dimensionality of each data point
* `step`: Granularity of skips. The lower the number, the noisier. The higher the number, the more erratic.

Take note we also capture the predicted character for the given sentence. This will allow us to setup the data in such a way that a given set of sequences predicts the next character. In machine learning, we will denote this as our `y` value or ground truth. Each `y` however is represented as a one hot encoding where a position will receive a value of `1` depending on the character position in `character_indices`

In [10]:
maxlen = 10
step = 15

sentences = []
next_characters = []

for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i+maxlen])
    next_characters.append(processed_text[i + maxlen])

In [11]:
sentences

['michael\nch',
 'pher\njessi',
 'tthew\nashl',
 'nnifer\njos',
 'manda\ndani',
 'vid\njames\n',
 't\njohn\njos',
 'ndrew\nryan',
 'don\njason\n',
 'n\nsarah\nwi',
 '\njonathan\n',
 'anie\nbrian',
 'le\nnichola',
 'hony\nheath',
 'ic\nelizabe',
 'am\nmegan\nm',
 'a\nkevin\nst',
 'thomas\ntim',
 'christina\n',
 'rachel\nlau',
 'uren\namber',
 'tany\ndanie',
 'ichard\nkim',
 '\njeffrey\na',
 'ystal\nmich',
 'tiffany\nje',
 'benjamin\nm',
 'mily\naaron',
 'les\nrebecc',
 'ob\nstephen',
 'ick\nsean\ne',
 'achary\njam',
 'lly\nsamant',
 'than\nsara\n',
 'n\npaul\nang',
 'yler\nscott',
 'erine\nandr',
 'egory\neric',
 'y\ntravis\nl',
 'enneth\nbry',
 'ndsey\nkris',
 'ose\nalexan',
 'esse\nkatie',
 'say\nshanno',
 'essa\ncourt',
 'hristine\na',
 '\ncody\nalli',
 'radley\nsam',
 'hawn\napril',
 'k\nkathryn\n',
 'in\nchad\nje',
 'ara\nmaria\n',
 'al\njared\na',
 'dward\njuli',
 'er\nholly\nm',
 '\nkristina\n',
 'ie\njordan\n',
 'ria\njacque',
 'corey\nkeit',
 'ica\njuan\nd',
 '\ncassandra',
 'a

In [12]:
next_characters

['r',
 'c',
 'e',
 'h',
 'e',
 'r',
 'e',
 '\n',
 'j',
 'l',
 's',
 '\n',
 's',
 'e',
 't',
 'e',
 'e',
 'o',
 'k',
 'r',
 '\n',
 'l',
 'b',
 'm',
 'e',
 'r',
 'a',
 '\n',
 'a',
 '\n',
 'r',
 'i',
 'h',
 'd',
 'e',
 '\n',
 'e',
 'a',
 'i',
 'a',
 't',
 'd',
 '\n',
 'n',
 'n',
 'l',
 's',
 'u',
 '\n',
 'k',
 'n',
 'k',
 'n',
 'e',
 'a',
 'n',
 'v',
 'l',
 'h',
 'o',
 '\n',
 'a',
 'r',
 'a',
 'e',
 'y',
 't',
 'c',
 'o',
 'l',
 'n',
 'e',
 '\n',
 'e',
 'u',
 'e',
 'i',
 '\n',
 't',
 'v',
 'm',
 'r',
 'b',
 'd',
 'n',
 'e',
 'e',
 'e',
 'a',
 'n',
 'e',
 'a',
 'e',
 'd',
 'n',
 'a',
 'a',
 'a',
 '\n',
 'e',
 's',
 'y',
 'i',
 's',
 'a',
 '\n',
 'l',
 't',
 'e',
 'l',
 'm',
 '\n',
 't',
 'u',
 't',
 'r',
 '\n',
 'b',
 'l',
 'r',
 'g',
 'a',
 'l',
 'a',
 'r',
 'n',
 'd',
 'o',
 'e',
 'g',
 '\n',
 'k',
 'n',
 'b',
 'm',
 '\n',
 'c',
 'y',
 's',
 'j',
 'e',
 'i',
 'n',
 'a',
 'k',
 'n',
 'm',
 'c',
 'd',
 'o',
 '\n',
 'l',
 'e',
 'e',
 'a',
 'g',
 't',
 'i',
 '\n',
 'a',
 'g',
 'u',
 'c',
 'c

### Vectorization

This step simply converts the `sentences` and `next_characters` to its `x` and `y` components respectively. Since we're using pytorch, we convert it to a tensor. Succeeding cells check the shape of `x` and `y`.

In [13]:
print("Vectorization")
# device = 'cuda'
device = 'cpu'
x = np.zeros((len(sentences), maxlen, len(characters)), dtype=np.float64)
y = np.zeros((len(sentences), len(characters)), dtype=np.float64)

for i, sentence in enumerate(sentences):
    for t, character, in enumerate(sentence):
        x[i, t, character_indices[character]] = 1
    y[i, character_indices[next_characters[i]]] = 1
    
x = torch.tensor(x).float().to(device)
y = torch.tensor(y).float().to(device)

Vectorization


In [14]:
x.shape

torch.Size([8904, 10, 27])

In [15]:
x = torch.flatten(x, start_dim=1)

x.shape

torch.Size([8904, 270])

In [16]:
y.shape

torch.Size([8904, 27])

### Utility Function for Generating Samples

In [17]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### Callback Function

This generates sample text from a given seed and ran for every epoch of the model.

In [18]:
def callback(model):
    start = 0
    stop = len(processed_text) - maxlen - 1

    #print("Start: {}".format(start))
    #print("Stop: {}".format(stop))

    start_index = random.randint(start, stop)

    #print("Start Index: {}".format(start_index))

    sentence = processed_text[start_index: start_index + maxlen]

    #print("Sentence: {}".format(sentence))
    #print("Sentence Length: {}".format(len(sentence)))

    generated = ''

    for i in range(400):
        x_predictions = np.zeros((1, maxlen, len(characters)))

        for t, char in enumerate(sentence):
            x_predictions[0, t, character_indices[char]] = 1

            # print(x_predictions)
        x_predictions = torch.tensor(x_predictions).float().to(device)
        x = torch.flatten(x_predictions, start_dim=1)

        preds = model.forward(x)[0].detach().cpu().numpy()

        next_index = sample(preds)
        #print("next_index: {}".format(next_index))
        next_char = indices_characters[next_index]
        #print("next_char: {}".format(next_char))

        generated += next_char
        sentence = sentence[1:] + next_char

    return sentence

### MultiLayerPerceptron Model

This will be our current language model. Although originally used for classification, we can also treat is a regression model since our ground truth represents the next predicted character in a sequence. Take note that this is a rather simplistic model without any properties to remember previous input. This forces the model to treat each input as an independent observation without considering sequential behavior. TLDR, it won't generate good results.

In [19]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.hidden = nn.Linear(input_dim, 500)
        self.output = nn.Linear(500, output_dim)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # f(x) = a(f(x))
        x = self.relu(self.hidden(x))
        y = self.sigmoid(self.output(x))

        return y

In [20]:
model = MultiLayerPerceptron(x.shape[1], y.shape[1]).to(device)

model

MultiLayerPerceptron(
  (hidden): Linear(in_features=270, out_features=500, bias=True)
  (output): Linear(in_features=500, out_features=27, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

### Training Function

In [21]:
optimizer = optim.Adam(model.parameters(), lr=0.00001)
criterion = nn.CrossEntropyLoss()

def train_fn(model, optimizer, loss_fn, device):
    ave_loss = 0
    count = 0
    
    for i, data in enumerate(x):
        data = x[i]
        targets = y[i]
        
        # Forward
        predictions = model.forward(data)
        
        predictions = F.softmax(predictions, dim=-1)
        
        loss = loss_fn(predictions, targets)
        
        # Backward
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()

        count += 1
        ave_loss += loss.item()
    
    ave_loss = ave_loss / count

    return ave_loss

epochs = 1000

average_losses = []

for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    ave_loss = train_fn(model, optimizer, criterion, device)
    
    average_losses.append(ave_loss)
        
    print("Ave Loss: {}".format(ave_loss))
    
    generated_sentence = callback(model)
    
    print("Generated sentence:")
    print(generated_sentence)
    print("Length: {}".format(len(generated_sentence)))

Epoch: 0
Ave Loss: 3.292993081895917
Generated sentence:
pzkmnsyupo
Length: 10
Epoch: 1
Ave Loss: 3.284796089228487
Generated sentence:
anrnnortra
Length: 10
Epoch: 2
Ave Loss: 3.2805232841049565
Generated sentence:
sron
i
ee

Length: 10
Epoch: 3
Ave Loss: 3.2793498745611522
Generated sentence:
iaiienle
i
Length: 10
Epoch: 4
Ave Loss: 3.2789249363399686
Generated sentence:
aearsiiee

Length: 10
Epoch: 5
Ave Loss: 3.2786696878297845
Generated sentence:
saarranean
Length: 10
Epoch: 6
Ave Loss: 3.2784256980907562
Generated sentence:



nii
ai

Length: 10
Epoch: 7
Ave Loss: 3.278147794246031
Generated sentence:
i
irsia

i
Length: 10
Epoch: 8
Ave Loss: 3.277827370306017
Generated sentence:
an
niahnrr
Length: 10
Epoch: 9
Ave Loss: 3.2774651421155356
Generated sentence:
naslaailir
Length: 10
Epoch: 10
Ave Loss: 3.27706812569823
Generated sentence:
i
eia
aes

Length: 10
Epoch: 11
Ave Loss: 3.2766471078346573
Generated sentence:
ianeaene
l
Length: 10
Epoch: 12
Ave Loss: 3.276231254662679
Genera

Ave Loss: 3.2651735010547673
Generated sentence:
henannanel
Length: 10
Epoch: 103
Ave Loss: 3.2651253775070086
Generated sentence:
rie
sanala
Length: 10
Epoch: 104
Ave Loss: 3.265076230478415
Generated sentence:

a
lierell
Length: 10
Epoch: 105
Ave Loss: 3.2650267372019957
Generated sentence:
eanea
lenn
Length: 10
Epoch: 106
Ave Loss: 3.264977735061423
Generated sentence:
inaana
shr
Length: 10
Epoch: 107
Ave Loss: 3.264929432057092
Generated sentence:
alerlinala
Length: 10
Epoch: 108
Ave Loss: 3.2648815444644677
Generated sentence:
hera
arias
Length: 10
Epoch: 109
Ave Loss: 3.2648334477200556
Generated sentence:
ndaanaran

Length: 10
Epoch: 110
Ave Loss: 3.2647846498847115
Generated sentence:
sl
manneri
Length: 10
Epoch: 111
Ave Loss: 3.2647356305535804
Generated sentence:
areithalla
Length: 10
Epoch: 112
Ave Loss: 3.2646873662413936
Generated sentence:
rlandia
li
Length: 10
Epoch: 113
Ave Loss: 3.2646405605293025
Generated sentence:
nerineal
k
Length: 10
Epoch: 114
Ave Loss: 3.2645950

Ave Loss: 3.2611148849009184
Generated sentence:
er
shanann
Length: 10
Epoch: 204
Ave Loss: 3.2610902864978
Generated sentence:
liea
sseni
Length: 10
Epoch: 205
Ave Loss: 3.2610658754771182
Generated sentence:
shellena
c
Length: 10
Epoch: 206
Ave Loss: 3.2610416186894797
Generated sentence:
rie
sera
k
Length: 10
Epoch: 207
Ave Loss: 3.26101740554765
Generated sentence:
allannel
l
Length: 10
Epoch: 208
Ave Loss: 3.260993437224
Generated sentence:
oeshil
eas
Length: 10
Epoch: 209
Ave Loss: 3.2609694661959163
Generated sentence:
llani
sane
Length: 10
Epoch: 210
Ave Loss: 3.2609457152176846
Generated sentence:
llena
shll
Length: 10
Epoch: 211
Ave Loss: 3.260922072657785
Generated sentence:
annianasha
Length: 10
Epoch: 212
Ave Loss: 3.26089854871809
Generated sentence:
n
annie
ne
Length: 10
Epoch: 213
Ave Loss: 3.2608751265025844
Generated sentence:
le
tannale
Length: 10
Epoch: 214
Ave Loss: 3.2608518598053977
Generated sentence:
n
senia
to
Length: 10
Epoch: 215
Ave Loss: 3.260828657720074


Ave Loss: 3.2591523268901863
Generated sentence:
erise
lero
Length: 10
Epoch: 305
Ave Loss: 3.2591362296731647
Generated sentence:
lea
lannis
Length: 10
Epoch: 306
Ave Loss: 3.2591201934263925
Generated sentence:
nel
lamar

Length: 10
Epoch: 307
Ave Loss: 3.2591041682918975
Generated sentence:
anilen
are
Length: 10
Epoch: 308
Ave Loss: 3.2590882065107665
Generated sentence:
earina
lan
Length: 10
Epoch: 309
Ave Loss: 3.2590722151683025
Generated sentence:
anianeen
t
Length: 10
Epoch: 310
Ave Loss: 3.2590563012369023
Generated sentence:

taronella
Length: 10
Epoch: 311
Ave Loss: 3.259040388992426
Generated sentence:
n
arannian
Length: 10
Epoch: 312
Ave Loss: 3.259024574937632
Generated sentence:
lieannel
e
Length: 10
Epoch: 313
Ave Loss: 3.2590087975667386
Generated sentence:
ielan
kall
Length: 10
Epoch: 314
Ave Loss: 3.2589930996419283
Generated sentence:
naelanne
l
Length: 10
Epoch: 315
Ave Loss: 3.2589774415071013
Generated sentence:
eresa
mane
Length: 10
Epoch: 316
Ave Loss: 3.258961

Ave Loss: 3.257731126673245
Generated sentence:
nisa
amare
Length: 10
Epoch: 406
Ave Loss: 3.2577189087921288
Generated sentence:
alae
saris
Length: 10
Epoch: 407
Ave Loss: 3.2577066938028825
Generated sentence:
era
enere

Length: 10
Epoch: 408
Ave Loss: 3.257694528939375
Generated sentence:
irean
alan
Length: 10
Epoch: 409
Ave Loss: 3.2576824130502136
Generated sentence:
halae
lari
Length: 10
Epoch: 410
Ave Loss: 3.2576703254371107
Generated sentence:
san
arasan
Length: 10
Epoch: 411
Ave Loss: 3.2576582703843164
Generated sentence:
arile
lara
Length: 10
Epoch: 412
Ave Loss: 3.2576462156260644
Generated sentence:

lena
leal
Length: 10
Epoch: 413
Ave Loss: 3.257634153504256
Generated sentence:
alin
teann
Length: 10
Epoch: 414
Ave Loss: 3.257622187670993
Generated sentence:
alena
leer
Length: 10
Epoch: 415
Ave Loss: 3.2576101744164245
Generated sentence:
ell
aranes
Length: 10
Epoch: 416
Ave Loss: 3.257598230191854
Generated sentence:
a
neristea
Length: 10
Epoch: 417
Ave Loss: 3.257586300

...

Ave Loss: 3.256253001503546
Generated sentence:
sharan
eri
Length: 10
Epoch: 537
Ave Loss: 3.2562423748468774
Generated sentence:
en
siarin

Length: 10
Epoch: 538
Ave Loss: 3.2562317873643307
Generated sentence:
el
jallia

Length: 10
Epoch: 539
Ave Loss: 3.256221201033177
Generated sentence:
ien
aralen
Length: 10
Epoch: 540
Ave Loss: 3.256210642442549
Generated sentence:
a
aranne
a
Length: 10
Epoch: 541
Ave Loss: 3.25620004537399
Generated sentence:
ania
alani
Length: 10
Epoch: 542
Ave Loss: 3.256189514122241
Generated sentence:

lereale
c
Length: 10
Epoch: 543
Ave Loss: 3.256178983432799
Generated sentence:
rinesha
na
Length: 10
Epoch: 544
Ave Loss: 3.256168526245042
Generated sentence:
rien
lanee
Length: 10
Epoch: 545
Ave Loss: 3.2561580386926545
Generated sentence:
a
larellia
Length: 10
Epoch: 546
Ave Loss: 3.256147573686139
Generated sentence:
n
arenne
l
Length: 10
Epoch: 547
Ave Loss: 3.2561371863048993
Generated sentence:
eeneal
ene
Length: 10
Epoch: 548
Ave Loss: 3.2561267664972

Ave Loss: 3.2552651015092744
Generated sentence:
senareanna
Length: 10
Epoch: 638
Ave Loss: 3.2552557556295008
Generated sentence:
lean
shila
Length: 10
Epoch: 639
Ave Loss: 3.2552464005385877
Generated sentence:
lea
alanae
Length: 10
Epoch: 640
Ave Loss: 3.2552370772046864
Generated sentence:
eera
senes
Length: 10
Epoch: 641
Ave Loss: 3.25522774969364
Generated sentence:

telaneel

Length: 10
Epoch: 642
Ave Loss: 3.2552183958612257
Generated sentence:
elsho
alee
Length: 10
Epoch: 643
Ave Loss: 3.2552090775613194
Generated sentence:
nesa
reree
Length: 10
Epoch: 644
Ave Loss: 3.2551997483901256
Generated sentence:
erica
male
Length: 10
Epoch: 645
Ave Loss: 3.2551904457812895
Generated sentence:

cenesha
n
Length: 10
Epoch: 646
Ave Loss: 3.255181106970531
Generated sentence:
res
callia
Length: 10
Epoch: 647
Ave Loss: 3.2551718065573736
Generated sentence:
a
manissel
Length: 10
Epoch: 648
Ave Loss: 3.2551624963172148
Generated sentence:
el
annesal
Length: 10
Epoch: 649
Ave Loss: 3.2551531

Ave Loss: 3.2543657376545565
Generated sentence:

lerisan
c
Length: 10
Epoch: 739
Ave Loss: 3.2543572237763656
Generated sentence:
la
carele

Length: 10
Epoch: 740
Ave Loss: 3.2543487277849237
Generated sentence:
lie
marlee
Length: 10
Epoch: 741
Ave Loss: 3.2543401713319864
Generated sentence:
n
alaneana
Length: 10
Epoch: 742
Ave Loss: 3.2543316550171277
Generated sentence:
anel
rella
Length: 10
Epoch: 743
Ave Loss: 3.254323079070848
Generated sentence:

arealanna
Length: 10
Epoch: 744
Ave Loss: 3.2543145207703272
Generated sentence:
ressin
ran
Length: 10
Epoch: 745
Ave Loss: 3.2543059449311538
Generated sentence:

emanael
m
Length: 10
Epoch: 746
Ave Loss: 3.254297401893278
Generated sentence:
lla
anna
l
Length: 10
Epoch: 747
Ave Loss: 3.2542888467791697
Generated sentence:

reresta
l
Length: 10
Epoch: 748
Ave Loss: 3.2542803148535704
Generated sentence:
tennet
ren
Length: 10
Epoch: 749
Ave Loss: 3.254271821004254
Generated sentence:
arissa
mer
Length: 10
Epoch: 750
Ave Loss: 3.2542633

Ave Loss: 3.25352534919522
Generated sentence:
r
elanna
e
Length: 10
Epoch: 840
Ave Loss: 3.2535176431661452
Generated sentence:
issa
lenas
Length: 10
Epoch: 841
Ave Loss: 3.2535099095839803
Generated sentence:
rina
lanis
Length: 10
Epoch: 842
Ave Loss: 3.253502203233587
Generated sentence:
ian
shiran
Length: 10
Epoch: 843
Ave Loss: 3.2534945551757866
Generated sentence:
en
kareena
Length: 10
Epoch: 844
Ave Loss: 3.253486895630837
Generated sentence:
ennit
jani
Length: 10
Epoch: 845
Ave Loss: 3.253479318986149
Generated sentence:
leena
eann
Length: 10
Epoch: 846
Ave Loss: 3.2534717576576586
Generated sentence:

anala
lea
Length: 10
Epoch: 847
Ave Loss: 3.2534642603519472
Generated sentence:
ien
jarell
Length: 10
Epoch: 848
Ave Loss: 3.253456806772374
Generated sentence:
sinn
ariss
Length: 10
Epoch: 849
Ave Loss: 3.2534493704904657
Generated sentence:
enasha
ten
Length: 10
Epoch: 850
Ave Loss: 3.2534419697410657
Generated sentence:
shanne
lar
Length: 10
Epoch: 851
Ave Loss: 3.2534346061

Ave Loss: 3.2527865416973643
Generated sentence:
lle
kamie

Length: 10
Epoch: 941
Ave Loss: 3.2527796943393357
Generated sentence:

aranneal

Length: 10
Epoch: 942
Ave Loss: 3.252772883986527
Generated sentence:
maria
shan
Length: 10
Epoch: 943
Ave Loss: 3.2527660778108634
Generated sentence:
nineea
ler
Length: 10
Epoch: 944
Ave Loss: 3.2527592768566307
Generated sentence:
elei
janne
Length: 10
Epoch: 945
Ave Loss: 3.252752488674822
Generated sentence:
marissa
ma
Length: 10
Epoch: 946
Ave Loss: 3.2527457155414456
Generated sentence:

sanna
sal
Length: 10
Epoch: 947
Ave Loss: 3.252738955260823
Generated sentence:
l
maliel
m
Length: 10
Epoch: 948
Ave Loss: 3.252732170372532
Generated sentence:
aen
lishea
Length: 10
Epoch: 949
Ave Loss: 3.2527254260239684
Generated sentence:
nnelle
jon
Length: 10
Epoch: 950
Ave Loss: 3.252718649650627
Generated sentence:
arina
kerr
Length: 10
Epoch: 951
Ave Loss: 3.25271186934113
Generated sentence:
onia
annel
Length: 10
Epoch: 952
Ave Loss: 3.25270510330